# **Training** 

#### Important!
Scripts ```exporter_main_v2.py & model_main_tf2.py```
should be run from withing the object_detection directory

In order to have a smooth training for our data, we have to install all related dependencies for our training process.

We will use **%%capture** below which is a simple way to suppress unwanted output.


### Installing dependencies


In [ ]:
# Installing dependencies
%%capture 

!pip install tensorflow-gpu
!pip install tf_slim
!pip install lvis
!pip install tensorflow_io
!pip install tf-models-official
!pip install --upgrade opencv-python
!pip install --upgrade opencv-contrib-python


### Cloning TensorFlow object detection 2.0 models from Github

In [ ]:
%%capture 

# !git clone https://github.com/tensorflow/models.git

In [ ]:
%cd /content/drive/Othercomputers/work_od/od/models/research/

/content/drive/Othercomputers/work_od/od/models/research


Tensorflow Object Detection API uses Protobufs to configure model and training parameters. Before the framework can be used, the Protobuf libraries must be downloaded and compiled.

In [ ]:
%%capture
!protoc object_detection/protos/*.proto --python_out=.


In TensorFlow 2.x, the pycocotools package is listed as a dependency of the Object Detection API. So we should get it installed when using the Object Detection API 

In [ ]:
%%capture
# !git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI
!make

In [ ]:
!cp -r pycocotools /content/drive/Othercomputers/work_od/od/models/research/

In [ ]:
%cd /content/drive/Othercomputers/work_od/od/models/research/

Installing the object_detection package.

In [ ]:
%%capture
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-feature=2020-resolver 


### Modifying our configuration file for the first model

We need to specify our learning rate, epochs, training path and other instructions in ```pipeline.config```  file for each model before the start of the actual training process 

When opening that file, there are many hyperparameters to manipulate and fine-tune for further improvements, but here we are focusing on the ones we need for our project.



Setting the number of classes (in our case we have 3)
```
 num_classes: 3 
 ```
Setting our batch_size (we used low number due to memory limit in Colab free resources)
 ```
batch_size: 8 
```

 
Setting the path to our desired model (SSD MobileNet_v2_320x320 OR efficientdet_d0_coco17_tpu_32) and specify the path to checkpoint of pre-trained model
```
fine_tune_checkpoint: "path_to_pre-trained-model/checkpoint/ckpt-0" 
```
setting the desired number of steps
```
num_steps: 2000
```

we will set this to "detection" instead of classification
```
fine_tune_checkpoint_type: "detection" 
```

the last two lines below will be set one time for train.record and one time for test.record
```
label_map_path: "annotations/label_map.pbtxt" # Path to label map file
input_path: "annotations/train.record" # Path to training TFRecord file
```


### Starting The Training 

After installing all necessary dependencies, we can now start our training

##### The first model: SSD MobileNet_v2_320x320

In [ ]:
#train mobilenet
%cd /content/drive/Othercomputers/work_od/od/rps_object_detection/
!python model_main_tf2.py --model_dir=/content/drive/Othercomputers/work_od/od/training_space/models_config/mobilenet --pipeline_config_path=/content/drive/Othercomputers/work_od/od/training_space/models_config/mobilenet/pipeline.config --alsologtostderr


2022-01-28 10:32:14.163729: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W0128 10:32:14.185370 140186872555392 cross_device_ops.py:1387] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0128 10:32:14.187294 140186872555392 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0128 10:32:15.111933 140186872555392 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0128 10:32:15.112276 140186872555392 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0128 10:32:15.220786 140186872555392 deprecation.py:347] From /content/dr

Exporting our trained model

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/drive/Othercomputers/work_od/od/training_space/models_config/mobilenet/pipeline.config --trained_checkpoint_dir /content/drive/Othercomputers/work_od/od/training_space/models_config/mobilenet --output_directory /content/drive/Othercomputers/work_od/od/training_space/exported_models/exported_mobilenet


2022-01-28 13:56:58.966115: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0128 13:56:59.670502 140205564520320 deprecation.py:619] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:464: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
INFO:tensorflow:depth of additional conv before box predictor: 0
I0128 13:57:07.691619 14

##### The Second model: Efficientdet_d0_coco17_tpu_32

In [ ]:
#train efficientdet_d0_coco17_tpu_32
!python model_main_tf2.py --model_dir=/content/drive/Othercomputers/work_od/od/training_space/models_config/efficientdet_d0_coco17_tpu_32 --pipeline_config_path=/content/drive/Othercomputers/work_od/od/training_space/models_config/efficientdet_d0_coco17_tpu_32/pipeline.config


2021-12-19 23:50:10.268738: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1219 23:50:10.272590 140272885966720 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I1219 23:50:10.995187 140272885966720 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1219 23:50:10.995453 140272885966720 config_util.py:552] Maybe overwriting use_bfloat16: False
I1219 23:50:11.006850 140272885966720 ssd_efficientnet_bifpn_feature_extractor.py:143] EfficientDet EfficientNet backbone version: efficientnet-b0
I1219 23:50:11.006979 140272885966720 ssd_efficientnet_bifpn_feature_extractor.py:144] 

In [ ]:
#export efficientdet_d0_coco17_tpu_32 
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/drive/Othercomputers/work_od/od/training_space/models_config/efficientdet_d0_coco17_tpu_32/pipeline.config --trained_checkpoint_dir /content/drive/Othercomputers/work_od/od/training_space/models_config/efficientdet_d0_coco17_tpu_32 --output_directory /content/drive/Othercomputers/work_od/od/training_space/exported_models/exported_efficientdet_d0_coco17_tpu_32


2022-01-03 05:34:26.208480: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I0103 05:34:26.254548 140308114024320 ssd_efficientnet_bifpn_feature_extractor.py:143] EfficientDet EfficientNet backbone version: efficientnet-b0
I0103 05:34:26.254778 140308114024320 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet BiFPN num filters: 64
I0103 05:34:26.254869 140308114024320 ssd_efficientnet_bifpn_feature_extractor.py:146] EfficientDet BiFPN num iterations: 3
I0103 05:34:26.258966 140308114024320 efficientnet_model.py:147] round_filter input=32 output=32
I0103 05:34:26.316798 140308114024320 efficientnet_model.py:147] round_filter input=32 output=32
I0103 05:34:26.316976 140308114024320 efficientnet_model.py:147] round_filter input=16 output=16
I0103 05:34:26.381664 140308114024320 efficientnet_model.py:147] round_filter input=16 output=16
I0103 05:34:26.381856 140308114024320 efficientnet_model.